# Chapter 19: Conditional Types

---

## 19.1 Introduction to Conditional Types

Conditional types allow you to create types that depend on a condition, selecting one of two possible types based on a type relationship test. They are one of TypeScript's most powerful features for type-level programming, enabling you to create dynamic, expressive type definitions that adapt based on the types they receive.

### 19.1.1 Conditional Type Syntax

Conditional types use a syntax similar to JavaScript's ternary operator, but operate at the type level:

```typescript
// Basic conditional type syntax
type ConditionalType = SomeType extends OtherType ? TrueType : FalseType;
```

The `extends` keyword checks if `SomeType` is assignable to `OtherType`. If it is, the type resolves to `TrueType`; otherwise, it resolves to `FalseType`.

**Basic Example:**

```typescript
// Check if a type is a string
type IsString<T> = T extends string ? "yes" : "no";

type A = IsString<"hello">;  // "yes" - literal type
type B = IsString<42>;       // "no" - literal type
type C = IsString<string>;   // "yes" - any string
type D = IsString<number>;   // "no" - any number

// Extract string types from a union
type ExtractString<T> = T extends string ? T : never;

type Mixed = string | number | boolean | "literal";
type OnlyStrings = ExtractString<Mixed>;  // string | "literal"
```

### 19.1.2 The `extends` Clause

The `extends` clause in conditional types works differently than in class inheritance. It performs a compatibility check: "Can type T be assigned to type U?"

```typescript
// Understanding extends in conditional types
type CheckExtends<T, U> = T extends U ? true : false;

// Primitive types
type T1 = CheckExtends<number, number>;      // true
type T2 = CheckExtends<42, number>;          // true (42 is assignable to number)
type T3 = CheckExtends<number, 42>;          // false (number is not assignable to 42)

// Object types
interface Animal { name: string; }
interface Dog extends Animal { breed: string; }

type T4 = CheckExtends<Dog, Animal>;         // true (Dog has all Animal properties)
type T5 = CheckExtends<Animal, Dog>;         // false (Animal lacks breed)

// Union types
type T6 = CheckExtends<string | number, string>;     // boolean (distributive)
type T7 = CheckExtends<string, string | number>;     // true

// any type
type T8 = CheckExtends<any, string>;         // boolean (special case: any extends anything)
type T9 = CheckExtends<string, any>;         // true
```

**Key Insight:** The `extends` check is structural, not nominal. TypeScript checks if the shape of the type satisfies the constraint.

```typescript
// Structural typing example
interface Point2D { x: number; y: number; }
interface Point3D { x: number; y: number; z: number; }

// Point3D extends Point2D structurally
type Is2D = Point3D extends Point2D ? true : false;  // true

// Even without explicit inheritance
const point3D: Point3D = { x: 1, y: 2, z: 3 };
const point2D: Point2D = point3D;  // Valid: Point3D has all Point2D properties
```

### 19.1.3 True and False Branches

Conditional types can resolve to any valid type in either branch, including complex types, unions, or even other conditional types.

```typescript
// Simple branches
type IsNumber<T> = T extends number ? T : never;

// Branching to different object shapes
type Transform<T> = T extends string 
  ? { kind: "string"; value: T; length: number }
  : T extends number
    ? { kind: "number"; value: T; doubled: number }
    : { kind: "other"; value: T };

type StringResult = Transform<"hello">;  
// { kind: "string"; value: "hello"; length: number }

type NumberResult = Transform<42>;       
// { kind: "number"; value: 42; doubled: number }

type BooleanResult = Transform<true>;    
// { kind: "other"; value: true }

// Nested conditional types
type DeepCheck<T> = T extends string
  ? T extends "admin" | "superuser"
    ? "high-privilege"
    : "normal-string"
  : T extends number
    ? T extends 0
      ? "zero"
      : "non-zero-number"
    : "other";

type D1 = DeepCheck<"admin">;      // "high-privilege"
type D2 = DeepCheck<"user">;       // "normal-string"
type D3 = DeepCheck<0>;            // "zero"
type D4 = DeepCheck<42>;           // "non-zero-number"
type D5 = DeepCheck<boolean>;      // "other"
```

**Practical Example - API Response Types:**

```typescript
// Conditional types for API responses
type ApiResponse<T, E = Error> = T extends void 
  ? { success: true; data: null }
  : { success: true; data: T }
  | { success: false; error: E };

// Usage
type UserResponse = ApiResponse<{ id: number; name: string }>;
// { success: true; data: { id: number; name: string } } | { success: false; error: Error }

type DeleteResponse = ApiResponse<void>;  
// { success: true; data: null } | { success: false; error: Error }

async function fetchUser(id: number): Promise<UserResponse> {
  try {
    const response = await fetch(`/api/users/${id}`);
    if (!response.ok) throw new Error("Failed to fetch");
    const data = await response.json();
    return { success: true, data };
  } catch (error) {
    return { 
      success: false, 
      error: error instanceof Error ? error : new Error(String(error)) 
    };
  }
}
```

---

## 19.2 Type Inference in Conditional Types

The `infer` keyword allows you to infer and capture types within the extends clause of a conditional type. This is one of the most powerful features for type manipulation, enabling you to extract types from complex structures.

### 19.2.1 The `infer` Keyword

`infer` declares a type variable that TypeScript will infer from the true branch of the condition.

```typescript
// Basic infer syntax
type ExtractType<T> = T extends Promise<infer U> ? U : T;

type PromiseString = Promise<string>;
type Extracted = ExtractType<PromiseString>;  // string

type JustNumber = ExtractType<number>;      // number (not a Promise, so returns T)

// Extract return type from function
type ReturnTypeOf<T> = T extends (...args: any[]) => infer R ? R : never;

function getUser() {
  return { id: 1, name: "John" };
}

type UserReturn = ReturnTypeOf<typeof getUser>;  
// { id: number; name: string }
```

**How `infer` Works:**

```typescript
// TypeScript analyzes the structure and infers the type
type ExtractArrayType<T> = T extends Array<infer U> ? U : never;

// When T is string[], TypeScript:
// 1. Checks if string[] extends Array<infer U>
// 2. Infers U as string
// 3. Returns string
type StringElement = ExtractArrayType<string[]>;  // string
type NumberElement = ExtractArrayType<number[]>;  // number
type NotArray = ExtractArrayType<string>;         // never
```

### 19.2.2 Inferring from Function Types

Extracting types from function signatures is a common pattern used by TypeScript's built-in utility types.

```typescript
// Extract parameter types
type ParametersOf<T> = T extends (...args: infer P) => any ? P : never;

function createUser(name: string, age: number, isAdmin: boolean) {
  return { name, age, isAdmin };
}

type CreateUserParams = ParametersOf<typeof createUser>;  
// [name: string, age: number, isAdmin: boolean]

// Extract specific parameter
type FirstParameter<T> = T extends (first: infer P, ...args: any[]) => any 
  ? P 
  : never;

type FirstParam = FirstParameter<typeof createUser>;  // string

// Extract function type from method
type MethodType<T, K extends keyof T> = T[K] extends (...args: any[]) => infer R
  ? (...args: ParametersOf<T[K]>) => R
  : never;

interface UserService {
  getUser(id: number): Promise<{ id: number; name: string }>;
  saveUser(user: { name: string }): Promise<void>;
}

type GetUserMethod = MethodType<UserService, "getUser">;
// (id: number) => Promise<{ id: number; name: string }>
```

### 19.2.3 Inferring from Array Types

Arrays and tuples provide rich opportunities for type inference.

```typescript
// Extract element type from array
type ElementType<T> = T extends (infer U)[] ? U : never;

type StringArray = string[];
type StringElement = ElementType<StringArray>;  // string

// Extract from tuple
type Tuple = [string, number, boolean];
type First = Tuple extends [infer H, ...any[]] ? H : never;      // string
type Rest = Tuple extends [any, ...infer T] ? T : never;         // [number, boolean]
type Last = Tuple extends [...any[], infer L] ? L : never;       // boolean

// Extract specific positions
type Second<T extends any[]> = T extends [any, infer S, ...any[]] ? S : never;
type SecondElement = Second<Tuple>;  // number

// Reverse a tuple
type Reverse<T extends any[]> = T extends [infer H, ...infer T]
  ? [...Reverse<T>, H]
  : [];

type Reversed = Reverse<[1, 2, 3]>;  // [3, 2, 1]
```

### 19.2.4 Multiple `infer` Positions

You can use multiple `infer` declarations to extract multiple types simultaneously.

```typescript
// Extract both key and value types from a mapped type
type KeyValue<T> = T extends { [K in infer Key]: infer Value }
  ? { key: Key; value: Value }
  : never;

// This doesn't work as expected because of how mapped types work
// Instead, use this approach:

// Extract from Record
type RecordInfo<T> = T extends Record<infer K, infer V> 
  ? { key: K; value: V }
  : never;

type UserRecord = Record<string, { name: string }>;
type UserRecordInfo = RecordInfo<UserRecord>;  
// { key: string; value: { name: string } }

// Extract from a specific structure
type AsyncFunctionInfo<T> = T extends (...args: infer Args) => Promise<infer Result>
  ? { args: Args; result: Result }
  : never;

async function fetchData(id: number, includeMeta: boolean): Promise<{ data: string }> {
  return { data: "result" };
}

type FetchInfo = AsyncFunctionInfo<typeof fetchData>;
// { args: [id: number, includeMeta: boolean]; result: { data: string } }

// Extract constructor parameters and instance type
type ConstructorInfo<T> = T extends new (...args: infer Args) => infer Instance
  ? { args: Args; instance: Instance }
  : never;

class User {
  constructor(public name: string, public age: number) {}
}

type UserConstructor = ConstructorInfo<typeof User>;
// { args: [name: string, age: number]; instance: User }
```

**Advanced Pattern - Recursive Type Extraction:**

```typescript
// Deep extraction through nested structures
type DeepPromiseValue<T> = T extends Promise<infer U>
  ? DeepPromiseValue<U>
  : T;

type NestedPromise = Promise<Promise<Promise<string>>>;
type Unwrapped = DeepPromiseValue<NestedPromise>;  // string

// Extract all function return types in a chain
type ChainReturns<T> = T extends (arg: any) => infer R
  ? R extends (arg: any) => any
    ? ChainReturns<R>
    : R
  : T;

const chain = (x: number) => (y: string) => (z: boolean) => ({ x, y, z });
type FinalReturn = ChainReturns<typeof chain>;  // { x: number; y: string; z: boolean }
```

---

## 19.3 Distributive Conditional Types

When conditional types act on a generic type parameter, they become *distributive* over union types. This is one of the most important behaviors to understand when working with conditional types.

### 19.3.1 Understanding Distribution

A conditional type is distributive when the checked type is a naked type parameter (not wrapped in another type).

```typescript
// Distributive behavior
type ToArray<T> = T extends any ? T[] : never;

// When T is a union, it distributes over each member
type StringOrNumber = string | number;
type ArrayOfEach = ToArray<StringOrNumber>;  
// string[] | number[] (NOT (string | number)[])

// Visual explanation:
// ToArray<string | number>
// = ToArray<string> | ToArray<number>
// = string[] | number[]
```

**Why Distribution Happens:**

```typescript
// Distribution occurs because TypeScript treats:
type Distributed = (string extends any ? string[] : never) 
                | (number extends any ? number[] : never);

// Which evaluates to:
// string[] | number[]
```

### 19.3.2 When Distribution Happens

Distribution only occurs when the checked type is a naked type parameter.

```typescript
// ✅ Distributive: Naked type parameter
type Distributive<T> = T extends any ? T[] : never;
type DistResult = Distributive<string | number>;  // string[] | number[]

// ❌ Non-distributive: Wrapped type parameter
type NonDistributive<T> = [T] extends [any] ? T[] : never;
type NonDistResult = NonDistributive<string | number>;  // (string | number)[]

// Other wrapping examples that prevent distribution
type Wrapped1<T> = { value: T } extends { value: any } ? T[] : never;
type Wrapped2<T> = (() => T) extends (() => any) ? T[] : never;

type W1 = Wrapped1<string | number>;  // (string | number)[]
type W2 = Wrapped2<string | number>;  // (string | number)[]
```

**Practical Implications:**

```typescript
// Use distribution to filter unions
type ExtractByType<T, U> = T extends U ? T : never;

type Mixed = string | number | boolean | string[];
type OnlyStrings = ExtractByType<Mixed, string>;  // string (not string[])

// Without distribution, we'd get:
type WrongExtract<T, U> = [T] extends [U] ? T : never;
type WrongResult = WrongExtract<Mixed, string>;  
// never (because [string | number | boolean | string[]] doesn't extend [string])

// Exclude types (opposite of Extract)
type ExcludeType<T, U> = T extends U ? never : T;
type NoStrings = ExcludeType<Mixed, string>;  // number | boolean | string[]

// Built-in utility types use this pattern:
// type Extract<T, U> = T extends U ? T : never;
// type Exclude<T, U> = T extends U ? never : T;
```

### 19.3.3 Preventing Distribution

Sometimes you want to check the union as a whole, not distribute over it.

```typescript
// Technique 1: Wrap both sides in tuples
type NoDistribute<T, U> = [T] extends [U] ? T : never;

type Check = NoDistribute<string | number, string>;  
// never (the whole union doesn't extend string)

// Technique 2: Use intersection
type NoDistribute2<T, U> = T & {} extends U ? T : never;

// Technique 3: Check within a function (creates a scope)
type NoDistribute3<T, U> = (T extends U ? T : never) extends infer R ? R : never;

// Practical use: Ensure exact type match
type Exact<A, B> = [A] extends [B] 
  ? [B] extends [A] 
    ? true 
    : false 
  : false;

type E1 = Exact<string, string>;           // true
type E2 = Exact<string | number, string>;  // false
type E3 = Exact<"a" | "b", string>;       // false (specific strings vs any string)
```

**Common Pattern - Type Predicates:**

```typescript
// Create type predicates using conditional types
type IsString<T> = T extends string ? true : false;

// Distributive version (checks each member)
type CheckUnion = IsString<string | number | "literal">;  // boolean

// Non-distributive version (checks the union as a whole)
type IsUnionString<T> = [T] extends [string] ? true : false;
type CheckUnion2 = IsUnionString<string | number>;  // false

// Check if type is exactly a specific type (not a subtype)
type IsExactType<T, U> = [T] extends [U] 
  ? [U] extends [T] 
    ? true 
    : false 
  : false;

type ExactString = IsExactType<string, string>;       // true
type LiteralString = IsExactType<"a", string>;       // false ("a" is more specific)
type SubtypeCheck = IsExactType<"a" | "b", string>;  // false
```

---

## 19.4 Built-in Conditional Types

TypeScript provides several built-in utility types that leverage conditional types. Understanding how these work helps you build your own custom utilities.

### 19.4.1 `Exclude<T, U>`

Removes types from `T` that are assignable to `U`.

```typescript
// Implementation
type Exclude<T, U> = T extends U ? never : T;

// Usage
type T0 = Exclude<"a" | "b" | "c", "a">;      
// "b" | "c"

type T1 = Exclude<"a" | "b" | "c", "a" | "b">;  
// "c"

type T2 = Exclude<string | number | (() => void), Function>;  
// string | number

// Practical example: Remove specific event types
type Event = 
  | { type: "click"; x: number; y: number }
  | { type: "focus"; element: HTMLElement }
  | { type: "blur"; element: HTMLElement };

type ClickEvent = Exclude<Event, { type: "focus" | "blur" }>;
// { type: "click"; x: number; y: number }
```

### 19.4.2 `Extract<T, U>`

Extracts types from `T` that are assignable to `U` (opposite of Exclude).

```typescript
// Implementation
type Extract<T, U> = T extends U ? T : never;

// Usage
type T0 = Extract<"a" | "b" | "c", "a" | "f">;  
// "a"

type T1 = Extract<string | number | (() => void), Function>;  
// (() => void)

// Practical example: Get only specific properties
interface User {
  name: string;
  age: number;
  email: string;
  password: string;
}

type PublicKeys = "name" | "email";
type PublicUser = Extract<keyof User, PublicKeys>;  // "name" | "email"

// Extract matching function overloads
type EventHandler = 
  | ((event: MouseEvent) => void)
  | ((event: KeyboardEvent) => void)
  | ((event: TouchEvent) => void);

type MouseHandler = Extract<EventHandler, (event: MouseEvent) => void>;
// (event: MouseEvent) => void
```

### 19.4.3 `NonNullable<T>`

Removes `null` and `undefined` from `T`.

```typescript
// Implementation
type NonNullable<T> = T extends null | undefined ? never : T;

// Usage
type T0 = NonNullable<string | number | undefined>;  
// string | number

type T1 = NonNullable<string[] | null | undefined>;  
// string[]

// Practical example: Safe property access
interface User {
  name: string | null;
  email?: string;
}

type NonNullProps<T> = {
  [K in keyof T]: NonNullable<T[K]>;
};

// Note: This doesn't work as expected because NonNullable is distributive
// Better approach:
type Defined<T> = T extends undefined ? never : T;
type NonNullUser = {
  [K in keyof User]: Defined<User[K]>;
};
// { name: string | null; email: string }
```

### 19.4.4 `ReturnType<T>`

Extracts the return type of a function type.

```typescript
// Implementation
type ReturnType<T extends (...args: any) => any> = 
  T extends (...args: any) => infer R ? R : any;

// Usage
declare function f1(): { a: number; b: string };

type T0 = ReturnType<() => string>;  
// string

type T1 = ReturnType<(s: string) => void>;  
// void

type T2 = ReturnType<<T>() => T>;  
// unknown

type T3 = ReturnType<typeof f1>;  
// { a: number; b: string }

// Practical example: Type-safe API calls
async function fetchUser(id: number) {
  return { id, name: "John", email: "john@example.com" };
}

async function fetchOrder(id: number) {
  return { id, total: 100, items: [] };
}

type UserData = ReturnType<typeof fetchUser>;  // Promise<{ id: number; ... }>
type OrderData = ReturnType<typeof fetchOrder>; // Promise<{ id: number; ... }>

// Unwrap the Promise
type UnwrappedUser = ReturnType<typeof fetchUser> extends Promise<infer U> 
  ? U 
  : never;
// { id: number; name: string; email: string }
```

### 19.4.5 `Parameters<T>`

Extracts the parameter types of a function type as a tuple.

```typescript
// Implementation
type Parameters<T extends (...args: any) => any> = 
  T extends (...args: infer P) => any ? P : never;

// Usage
declare function f1(arg: { a: number; b: string }): void;

type T0 = Parameters<() => string>;  
// []

type T1 = Parameters<(s: string) => void>;  
// [s: string]

type T2 = Parameters<<T>(arg: T) => T>;  
// [arg: unknown]

type T3 = Parameters<typeof f1>;  
// [arg: { a: number; b: string }]

// Practical example: Type-safe wrapper functions
function logWrapper<T extends (...args: any[]) => any>(
  fn: T,
  ...args: Parameters<T>
): ReturnType<T> {
  console.log("Calling function with:", args);
  return fn(...args);
}

function greet(name: string, age: number) {
  return `Hello ${name}, you are ${age}`;
}

const result = logWrapper(greet, "John", 30);  // Type-safe!
// "Hello John, you are 30"
```

### 19.4.6 `InstanceType<T>`

Extracts the instance type of a constructor function type.

```typescript
// Implementation
type InstanceType<T extends abstract new (...args: any) => any> = 
  T extends abstract new (...args: any) => infer R ? R : any;

// Usage
class C {
  x = 0;
  y = 0;
}

type T0 = InstanceType<typeof C>;  
// C

type T1 = InstanceType<any>;  
// any

type T2 = InstanceType<never>;  
// never

// Practical example: Factory pattern
interface ServiceConstructor<T> {
  new (...args: any[]): T;
}

function createService<T>(Ctor: ServiceConstructor<T>): T {
  return new Ctor();
}

class UserService {
  users: string[] = [];
  getUsers() { return this.users; }
}

class OrderService {
  orders: number[] = [];
  getOrders() { return this.orders; }
}

const userService = createService(UserService);  // Type: UserService
const orderService = createService(OrderService); // Type: OrderService
```

**Additional Built-in Types:**

```typescript
// Awaited<T> - Unwraps Promise types
type Awaited<T> = T extends PromiseLike<infer U> ? Awaited<U> : T;

type A = Awaited<Promise<string>>;  // string
type B = Awaited<Promise<Promise<number>>>;  // number

// Uppercase, Lowercase, Capitalize, Uncapitalize (template literal types)
type Upper<T extends string> = Uppercase<T>;
type Lower<T extends string> = Lowercase<T>;

type Hello = Upper<"hello">;  // "HELLO"
type World = Capitalize<"world">;  // "World"

// ThisParameterType, OmitThisParameter (for functions with this)
type ThisParam<T> = T extends (this: infer U, ...args: any[]) => any ? U : unknown;

function clickHandler(this: HTMLElement, event: MouseEvent) {
  console.log(this.tagName);
}

type ThisType = ThisParam<typeof clickHandler>;  // HTMLElement
```

---

## 19.5 Advanced Conditional Type Patterns

Conditional types unlock sophisticated type manipulation patterns used in advanced TypeScript libraries and frameworks.

### 19.5.1 Recursive Conditional Types

Conditional types can be recursive, allowing deep type traversal.

```typescript
// Deep partial type
type DeepPartial<T> = T extends object 
  ? { [K in keyof T]?: DeepPartial<T[K]> }
  : T;

interface User {
  id: number;
  profile: {
    name: string;
    address: {
      street: string;
      city: string;
    };
  };
}

type PartialUser = DeepPartial<User>;
// All properties become optional at every level

// Deep readonly
type DeepReadonly<T> = T extends object 
  ? { readonly [K in keyof T]: DeepReadonly<T[K]> }
  : T;

// Deep required
type DeepRequired<T> = T extends object 
  ? { [K in keyof T]-?: DeepRequired<T[K]> }
  : T;

// Flatten nested arrays
type Flatten<T> = T extends Array<infer U> 
  ? Flatten<U> 
  : T;

type Nested = Flatten<string[][][]>;  // string

// Get all paths in an object (for type-safe accessors)
type Paths<T> = T extends object 
  ? { [K in keyof T]: K extends string 
      ? T[K] extends object 
        ? `${K}` | `${K}.${Paths<T[K]>}`
        : `${K}`
      : never
    }[keyof T]
  : never;

interface Config {
  database: {
    host: string;
    port: number;
  };
  api: {
    version: string;
    endpoints: {
      users: string;
      orders: string;
    };
  };
}

type ConfigPaths = Paths<Config>;
// "database" | "database.host" | "database.port" | "api" | "api.version" | "api.endpoints" | ...
```

### 19.5.2 Type Guards with Conditional Types

Combine conditional types with type guards for runtime type safety.

```typescript
// Define type guards using conditional types
type TypeGuard<T, U extends T> = (value: T) => value is U;

// Create a type guard factory
function createTypeGuard<T extends string>(type: T): TypeGuard<string, T> {
  return (value: string): value is T => value === type;
}

const isSuccess = createTypeGuard("success");
const isError = createTypeGuard("error");

function handleResult(result: string) {
  if (isSuccess(result)) {
    console.log("Success!");  // result is typed as "success"
  } else if (isError(result)) {
    console.log("Error!");    // result is typed as "error"
  }
}

// Conditional type for narrowing
type Narrowed<T, U> = T extends U ? T : never;

function narrow<T, U extends T>(value: T, guard: (v: T) => v is U): U {
  if (guard(value)) return value;
  throw new Error("Type mismatch");
}

// Branded types with conditional checks
type Brand<T, B> = T & { __brand: B };

type UserId = Brand<number, "UserId">;
type OrderId = Brand<number, "OrderId">;

function createUserId(id: number): UserId {
  return id as UserId;
}

function createOrderId(id: number): OrderId {
  return id as OrderId;
}

// Conditional type to extract brand
type ExtractBrand<T> = T extends Brand<any, infer B> ? B : never;
type IdType = ExtractBrand<UserId>;  // "UserId"
```

### 19.5.3 Mapped Types with Conditionals

Combine mapped types and conditional types for powerful transformations.

```typescript
// Conditional mapped type
type ConditionalPick<T, U> = {
  [K in keyof T as T[K] extends U ? K : never]: T[K];
};

interface User {
  id: number;
  name: string;
  age: number;
  email: string;
  metadata: Record<string, any>;
}

type StringProps = ConditionalPick<User, string>;
// { name: string; email: string }

// Filter by key pattern
type PickByPattern<T, Pattern extends string> = {
  [K in keyof T as K extends `${Pattern}${string}` ? K : never]: T[K];
};

interface Config {
  API_URL: string;
  API_KEY: string;
  DB_HOST: string;
  DB_PORT: number;
}

type ApiConfig = PickByPattern<Config, "API_">;
// { API_URL: string; API_KEY: string }

// Transform values conditionally
type TransformValues<T, From, To> = {
  [K in keyof T]: T[K] extends From ? To : T[K];
};

type NullableToOptional<T> = {
  [K in keyof T as T[K] extends null | undefined ? never : K]: T[K];
} & {
  [K in keyof T as T[K] extends null | undefined ? K : never]?: NonNullable<T[K]>;
};

interface FormData {
  name: string;
  email: string | null;
  phone: string | undefined;
}

type OptionalForm = NullableToOptional<FormData>;
// { name: string; email?: string; phone?: string }
```

### 19.5.4 Utility Type Composition

Build complex utilities by composing conditional types.

```typescript
// Deep Omit
type DeepOmit<T, K extends string> = T extends object 
  ? K extends `${infer P}.${infer Rest}`
    ? P extends keyof T
      ? { [Key in keyof T]: Key extends P ? DeepOmit<T[Key], Rest> : T[Key] }
      : T
    : Omit<T, K>
  : T;

interface Data {
  user: {
    profile: {
      name: string;
      age: number;
      password: string;
    };
    settings: {
      theme: string;
    };
  };
}

type WithoutPassword = DeepOmit<Data, "user.profile.password">;
// Removes password at nested path

// Strict Omit (ensures keys exist)
type StrictOmit<T, K extends keyof T> = Omit<T, K>;

// Merge types with conflict resolution
type Merge<A, B> = {
  [K in keyof A | keyof B]: K extends keyof B 
    ? B[K] 
    : K extends keyof A 
      ? A[K] 
      : never;
};

// Pick by value type
type PickByValue<T, V> = Pick<T, {
  [K in keyof T]: T[K] extends V ? K : never;
}[keyof T]>;

interface Employee {
  name: string;
  salary: number;
  department: string;
  isActive: boolean;
}

type NumericProps = PickByValue<Employee, number>;
// { salary: number }
```

### 19.5.5 Conditional Types for API Design

Real-world patterns for building type-safe APIs.

```typescript
// Type-safe event emitter
type EventMap = {
  userCreated: { id: number; name: string };
  userUpdated: { id: number; changes: Partial<{ name: string; email: string }> };
  userDeleted: { id: number };
};

type EventPayload<T extends keyof EventMap> = EventMap[T];

class TypedEventEmitter<Events extends Record<string, any>> {
  private listeners: { [K in keyof Events]?: Array<(payload: Events[K]) => void> } = {};

  on<K extends keyof Events>(event: K, listener: (payload: Events[K]) => void): void {
    if (!this.listeners[event]) {
      this.listeners[event] = [];
    }
    this.listeners[event]!.push(listener);
  }

  emit<K extends keyof Events>(event: K, payload: Events[K]): void {
    this.listeners[event]?.forEach(listener => listener(payload));
  }
}

const emitter = new TypedEventEmitter<EventMap>();

emitter.on("userCreated", (data) => {
  console.log(data.name);  // TypeScript knows data has id and name
});

// emitter.emit("userCreated", { id: 1 }); // Error: missing name

// Type-safe configuration builder
type ConfigBuilder<T = {}> = {
  with<K extends string, V>(key: K, value: V): ConfigBuilder<T & Record<K, V>>;
  build(): T;
};

function createConfig(): ConfigBuilder {
  return {
    with(key, value) {
      return {
        ...this,
        with: this.with,
        build: () => ({ ...this.build(), [key]: value }),
      } as ConfigBuilder<any>;
    },
    build() {
      return {} as any;
    },
  };
}

const config = createConfig()
  .with("apiUrl", "https://api.example.com")
  .with("timeout", 5000)
  .with("retries", 3)
  .build();

// config is typed as { apiUrl: string; timeout: number; retries: number }
```

---

## 19.6 Chapter Summary and Exercises

### Chapter Summary

In this chapter, we explored conditional types, one of TypeScript's most powerful features for type-level programming:

**Key Takeaways:**

1. **Conditional Type Basics**:
   - Syntax: `T extends U ? X : Y`
   - Uses structural typing to check assignability
   - Enables type-level logic and branching

2. **Type Inference with `infer`**:
   - Extract types from complex structures
   - Capture function return types, parameter types, and array element types
   - Multiple `infer` declarations for extracting multiple types

3. **Distributive Conditional Types**:
   - Automatically distribute over union types
   - Naked type parameters trigger distribution
   - Wrap in tuples to prevent distribution when needed

4. **Built-in Utility Types**:
   - `Exclude`, `Extract`, `NonNullable` for filtering
   - `ReturnType`, `Parameters`, `InstanceType` for introspection
   - All implemented using conditional types

5. **Advanced Patterns**:
   - Recursive types for deep transformations
   - Combining with mapped types for selective transformations
   - Building type-safe APIs and DSLs

### Practical Exercises

**Exercise 1: Basic Conditional Types**

Create the following conditional types:

```typescript
// 1. IsArray<T> - returns true if T is an array, false otherwise
// 2. IsPromise<T> - returns true if T is a Promise
// 3. UnwrapArray<T> - extracts the element type from an array
// 4. UnwrapPromise<T> - extracts the resolved type from a Promise
// 5. IsNullable<T> - returns true if T can be null or undefined

// Test cases:
type T1 = IsArray<string[]>;        // Should be true
type T2 = IsArray<string>;          // Should be false
type T3 = UnwrapPromise<Promise<number>>;  // Should be number
type T4 = IsNullable<string | null>; // Should be true
```

**Exercise 2: Type Extraction**

Implement types that extract information from functions and objects:

```typescript
// 1. FirstArgument<T> - extracts the type of the first argument
// 2. SecondArgument<T> - extracts the type of the second argument
// 3. HasProperty<T, K> - returns true if T has property K
// 4. PropertyType<T, K> - returns the type of property K in T (if it exists)

// Test cases:
declare function fn(a: string, b: number, c: boolean): void;
type A1 = FirstArgument<typeof fn>;  // Should be string
type A2 = SecondArgument<typeof fn>; // Should be number

interface User { name: string; age: number; }
type P1 = HasProperty<User, "name">;      // Should be true
type P2 = HasProperty<User, "email">;     // Should be false
type P3 = PropertyType<User, "age">;      // Should be number
```

**Exercise 3: Distributive Types**

Practice with distributive conditional types:

```typescript
// 1. NonNullableProperties<T> - makes all properties non-nullable
// 2. RequiredProperties<T, K> - makes specific keys required
// 3. OptionalProperties<T, K> - makes specific keys optional
// 4. PickByType<T, U> - picks only properties assignable to U

// Test cases:
interface Person {
  name: string;
  age: number | null;
  email?: string;
  phone: string | undefined;
}

type N = NonNullableProperties<Person>;  // age: number, phone: string
type R = RequiredProperties<Person, "email">; // email becomes required
type P = PickByType<Person, string | undefined>; // name, email, phone
```

**Exercise 4: Advanced Patterns**

Build complex utility types:

```typescript
// 1. DeepPartial<T> - makes all properties optional recursively
// 2. DeepReadonly<T> - makes all properties readonly recursively  
// 3. Flatten<T> - flattens nested arrays to single level
// 4. TupleToUnion<T> - converts a tuple to a union type
// 5. UnionToTuple<T> - converts a union to a tuple (advanced!)

// Test cases:
interface Nested {
  a: { b: { c: string; }; };
}

type D = DeepPartial<Nested>;  // All levels optional
type F = Flatten<string[][]>;  // Should be string[]

type U = TupleToUnion<[string, number, boolean]>;  // string | number | boolean
```

**Exercise 5: Real-World Application**

Build a type-safe state management system:

```typescript
// Define types for:
// 1. Action creators that return typed actions
// 2. Reducers that handle specific action types
// 3. Selectors that extract typed slices of state
// 4. Middleware that can intercept actions

// Requirements:
// - Actions should be discriminated unions with 'type' property
// - Reducer should enforce all actions are handled (exhaustiveness checking)
// - Selectors should be type-safe and composable
// - State updates should be immutable

interface State {
  users: { id: number; name: string }[];
  loading: boolean;
  error: string | null;
}

// Your implementation here
type Action = /* define action union */;

function reducer(state: State, action: Action): State {
  // Implement with exhaustiveness checking
}

function createSelector<T, R>(/* ... */): (state: T) => R {
  // Implementation
}
```

### Additional Resources

- **TypeScript Handbook - Conditional Types**: https://www.typescriptlang.org/docs/handbook/2/conditional-types.html
- **TypeScript Handbook - Type Inference in Conditional Types**: https://www.typescriptlang.org/docs/handbook/2/conditional-types.html#inferring-within-conditional-types
- **Total TypeScript by Matt Pocock**: https://www.totaltypescript.com/concepts
- **Type Challenges**: https://github.com/type-challenges/type-challenges (Practice conditional types with real problems)

---

## Coming Up Next: Chapter 20 - Template Literal Types

In the next chapter, we will explore **Template Literal Types**, one of TypeScript's most exciting features for string manipulation at the type level:

- **20.1 Introduction to Template Literal Types** - Creating types from string templates
- **20.2 String Manipulation Types** - Built-in utilities like `Uppercase`, `Lowercase`, `Capitalize`
- **20.3 Combining with Union Types** - Generating string literal unions dynamically
- **20.4 Template Literal Type Inference** - Extracting and manipulating string patterns
- **20.5 Real-World Use Cases** - Type-safe routing, event names, CSS properties, and more

Template literal types allow you to perform string operations at the type level, enabling incredibly expressive type definitions for APIs, routing systems, and domain-specific languages. Combined with conditional types, they form the foundation of TypeScript's advanced type system capabilities.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='18. mapped_types.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='20. template_literal_types.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
